# RadarX Fundamentals Demonstration

This notebook demonstrates the usage of all core functions in the `radarx.fundamentals` module.
We categorize the functions by domain: **Radar Constants**, **Geometry**, **Beam Properties**, **Power Calculations**,
**Doppler Properties**, **Timing**, **Reflectivity**, and **Scattering**. We will also **print** and **plot** outputs
for verification and provide scientific explanations alongside each step.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from radarx.fundamentals import *
from pprint import pprint

## 📌 Constants Check
We verify the radar constants.

In [ ]:
print(f"Speed of light (C): {constants.C} m/s")
print(f"Boltzmann constant (K): {constants.K_BOLTZMANN} J/K")

## 📡 Beam Characteristics
We examine how the radar beam spreads and calculate related parameters.

In [ ]:
# Horizontal and vertical beamwidth
beamwidth_deg = 1.0
range_m = 10_000
pulse_length = 150  # in meters

bw_rad = beam.beamwidth_to_radians(beamwidth_deg)
res_az = beam.azimuthal_resolution(range_m, beamwidth_deg)
vol = beam.volume_resolution(range_m, beamwidth_deg, beamwidth_deg, pulse_length)

print(f"Beamwidth (radians): {bw_rad:.6f}")
print(f"Azimuthal resolution at 10 km: {res_az:.2f} m")
print(f"Sample volume at 10 km: {vol:.2f} m³")

## ⚡ Power Calculations
We compute peak power, average power, and minimum detectable signal.

In [ ]:
pk_power = power.compute_peak_power(1000, 50)
avg_power = power.compute_average_power(pk_power, 0.1)
min_signal = power.compute_min_detectable_signal(1e6, 290)

print(f"Peak Power: {pk_power:.2f} W")
print(f"Average Power (10% duty): {avg_power:.2f} W")
print(f"Minimum Detectable Signal: {min_signal:.2e} W")

## 🌪️ Doppler Radar Parameters
Nyquist velocity and unambiguous range are calculated based on PRF and wavelength.

In [ ]:
prf = 1000  # Hz
wavelength = 0.03  # 3 cm

v_nyq = doppler.nyquist_velocity(prf, wavelength)
r_unamb = doppler.unambiguous_range(prf)
v_dual = doppler.dual_prf_velocity(wavelength, 1000, 800)

print(f"Nyquist Velocity: {v_nyq:.2f} m/s")
print(f"Unambiguous Range: {r_unamb:.2f} m")
print(f"Dual PRF Velocity: {v_dual:.2f} m/s")

## ⏱️ Timing Parameters

In [ ]:
pw = 1e-6  # 1 microsecond
duty = timing.compute_duty_cycle(pw, prf)
blind = timing.compute_blind_range(pw)

print(f"Duty Cycle: {duty:.3f}")
print(f"Blind Range: {blind:.2f} m")

## 📈 Visualization of Beam Cross Section

In [ ]:
# Plot azimuthal beam spread
theta = np.linspace(-bw_rad, bw_rad, 500)
r = np.full_like(theta, range_m)
x = r * np.sin(theta)
y = r * np.cos(theta)

plt.figure(figsize=(6, 6))
plt.plot(x, y)
plt.title("Azimuthal Beam Spread at 10 km")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.grid(True)
plt.axis("equal")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Radar beam specs
range_m = 10_000
beamwidth_deg = 1.0
bw_rad = np.deg2rad(beamwidth_deg)

# Define edges
theta_edges = np.linspace(-bw_rad / 2, bw_rad / 2, 300)
r_edges = np.linspace(0, range_m, 200)

# Create 2D meshgrid for plotting
Theta_edges, R_edges = np.meshgrid(theta_edges, r_edges)
X = R_edges * np.sin(Theta_edges)
Y = R_edges * np.cos(Theta_edges)

# Create dummy field (like reflectivity pattern or just theta for illustration)
C = Theta_edges[:-1, :-1]  # Trim to match the cell interior

# Plot using pcolormesh
plt.figure(figsize=(6, 6))
pc = plt.pcolormesh(X, Y, C, shading="auto", cmap="viridis")
plt.title("Radar Beam Cross Section (Azimuthal Plane)")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
# plt.axis('equal')
plt.xlim(-500, 500)
plt.grid(True)
plt.colorbar(pc, label="Angle (rad)")
plt.show()

In [ ]:
beamwidth_deg = 1.0
beamwidth_rad = np.deg2rad(beamwidth_deg)
beamwidth_rad_std = beamwidth_rad / (2 * np.sqrt(2 * np.log(2)))
I = np.exp(-((theta / beamwidth_rad_std) ** 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
beamwidth_deg = 1.0
beamwidth_rad = np.deg2rad(beamwidth_deg)
beamwidth_rad_std = beamwidth_rad / (2 * np.sqrt(2 * np.log(2)))

range_m = np.linspace(0, 10000, 500)
theta = np.linspace(-beamwidth_rad, beamwidth_rad, 300)

# Create meshgrid
R, Theta = np.meshgrid(range_m, theta, indexing="ij")  # shape: (500, 300)

# Convert polar to Cartesian
X = R * np.sin(Theta)
Y = R * np.cos(Theta)

# Gaussian beam intensity
I = np.exp(-((Theta / beamwidth_rad_std) ** 2))

# Plot
plt.figure(figsize=(6, 6))
plt.pcolormesh(X, Y, I, shading="nearest", cmap="inferno")
plt.colorbar(label="Normalized Beam Intensity")
plt.title("Radar Beam Cross Section (Azimuthal Gaussian Profile)")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.axis("equal")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Compute bin edges for range and angle
r_edges = 0.5 * (range_m[:-1] + range_m[1:])
r_edges = np.concatenate(
    [
        [range_m[0] - (r_edges[0] - range_m[0])],
        r_edges,
        [range_m[-1] + (range_m[-1] - r_edges[-1])],
    ]
)

theta_edges = 0.5 * (theta[:-1] + theta[1:])
theta_edges = np.concatenate(
    [
        [theta[0] - (theta_edges[0] - theta[0])],
        theta_edges,
        [theta[-1] + (theta[-1] - theta_edges[-1])],
    ]
)

# 2D edges
R_edges, Theta_edges = np.meshgrid(r_edges, theta_edges, indexing="ij")
X_edges = R_edges * np.sin(Theta_edges)
Y_edges = R_edges * np.cos(Theta_edges)

# Plot without warning
plt.figure(figsize=(6, 6))
plt.pcolormesh(X_edges, Y_edges, I, shading="auto", cmap="inferno")
plt.colorbar(label="Normalized Beam Intensity")
plt.title("Radar Beam Cross Section (Azimuthal Gaussian Profile)")
plt.xlabel("X (m)")
plt.ylabel("Y (m)")
plt.axis("equal")
plt.grid(True)
plt.tight_layout()
plt.show()